In [222]:
import pandas as pd
import numpy as np
import random
from random import randrange

In [223]:
# create the relevant dataframes
df_geodaten = pd.read_csv("Orte Schweiz.csv")
df_geodaten.PLZ = df_geodaten.PLZ.astype(int)

df_emaildomains = pd.read_csv("Emaildomains - Emaildomains.csv")
df_vornamen_frauen = pd.read_csv("Vornamen_Frauen - Sheet1.csv")
df_vornamen_männer = pd.read_csv("Vornamen_Männer - Sheet1.csv")
df_nachnamen = pd.read_csv("Nachnamen - Nachnamen.csv")

# Import all the streetnames given
df_strassen_zh = pd.read_csv("Strassennamen_Zurich.csv", names=['Strassennamen'])
df_strassen_be = pd.read_csv("Strassennamen_Bern.csv", names=['Strassennamen'])
df_strassen_bs = pd.read_csv("Strassennamen_Basel.csv", names=['Strassennamen'])
df_strassen_lu = pd.read_csv("Strassennamen_Luzern.csv", names=['Strassennamen'])
df_strassen_ge = pd.read_csv("Strassennamen_Genève.csv", names=['Strassennamen'])
df_strassen_sg = pd.read_csv("Strassennamen_St.Gallen.csv", names=['Strassennamen'])

In [224]:
#limits that you want to use in terms of output
limit_datarecords = 1000000 # how many fictional people do we want to create
street_range = 200
city_filter = ["Zürich", "Bern", "Luzern", "Basel", "Genève", "St. Gallen"] #cities we use (for now)

In [225]:
# Manipulations to get certain lists needed for this
list_emaildomains = df_emaildomains["Domain"].tolist()

In [226]:
# Filter the dataframe to the four mentioned cities
df_filtered_geodata = df_geodaten[(df_geodaten["Gemeindename"]==city_filter[0])
                                | (df_geodaten["Gemeindename"]==city_filter[1])
                                | (df_geodaten["Gemeindename"]==city_filter[2])
                                | (df_geodaten["Gemeindename"]==city_filter[3])
                                | (df_geodaten["Gemeindename"]==city_filter[4])
                                | (df_geodaten["Gemeindename"]==city_filter[5])]

In [227]:
number_lines = df_filtered_geodata.shape[0]
repeater = int(round((limit_datarecords/number_lines), 0))

In [228]:
# combine streets 
filtered_orte = set(df_filtered_geodata.Ort.tolist())
filtered_plz = set(df_filtered_geodata.PLZ.tolist())

# combine all street names into a large list 
list_strassen = []
filtered_streets_df = [df_strassen_zh,
                    df_strassen_be,
                    df_strassen_bs,
                    df_strassen_lu,
                    df_strassen_ge,
                    df_strassen_sg]

for frame in filtered_streets_df:
    source_list = df_strassen_zh["Strassennamen"].tolist()
    for strasse in source_list:
        list_strassen.append(strasse)

In [229]:
liste_vornamen = []

liste_nachnamen = df_nachnamen["Nachname"].tolist()
list_frau_vornamen = df_vornamen_frauen["Vorname"].tolist()
list_mann_vornamen = df_vornamen_männer["Vorname"].tolist()

for vorname in list_frau_vornamen:
    liste_vornamen.append(vorname)
    
for vorname in list_mann_vornamen:
    liste_vornamen.append(vorname)

# shuffles the names
random.shuffle(liste_vornamen)
random.shuffle(liste_nachnamen)

len_vornamen = len(liste_vornamen)
len_nachnamen = len(liste_nachnamen)

In [230]:
# copies the dataframe to keep all the data inside (later shuffling will take place)
df_raw_source = pd.concat([df_filtered_geodata]*repeater, ignore_index=True)

In [231]:
print("Der Dataframe hat nun: ", df_raw_source.shape[0], "Zeilen")
print("Wir haben: ", len(list_strassen), "Strassennamen")
print("Vornamen: ", len(liste_vornamen))
print("Nachnamen: ", len(liste_nachnamen))

Der Dataframe hat nun:  999999 Zeilen
Wir haben:  12864 Strassennamen
Vornamen:  1997
Nachnamen:  22571


In [232]:
# variables for telephone numbers
vorwahl = list(range(76,79))# carrier prefix adjust to your liking
Hauptnummer = list(range(2500049, 9999999)) # actual phone number range, adjust to your liking

In [233]:
df_list_vornamen = []
df_list_nachnamen = []
df_list_strassen = []
df_list_hausnummern = []
df_list_emaildomains = []
df_list_telefonnummer = []

length_loop = df_raw_source.shape[0]

In [234]:
#shuffle both first and lastname + streets
for datapoint in range(0, length_loop):
    df_list_vornamen.append(random.choice(liste_vornamen))
    df_list_nachnamen.append(random.choice(liste_nachnamen))
    df_list_strassen.append(random.choice(list_strassen))
    df_list_hausnummern.append(randrange(street_range))
    df_list_emaildomains.append(random.choice(list_emaildomains))
    df_list_telefonnummer.append("0" + str(random.choice(vorwahl)) + "/" + str(random.choice(Hauptnummer)))
    
    
print("Länge Vornamen: ", len(df_list_vornamen))
print("Länge Nachnamen: ", len(df_list_nachnamen))
print("Länge Strassen: ", len(df_list_vornamen))
print("Länge Hausnummern: ", len(df_list_vornamen))
print("Länge Emaildomains: ", len(df_list_emaildomains))
print("Länge Telefonnummeren: ", len(df_list_telefonnummer))

Länge Vornamen:  999999
Länge Nachnamen:  999999
Länge Strassen:  999999
Länge Hausnummern:  999999
Länge Emaildomains:  999999
Länge Telefonnummeren:  999999


In [235]:
outputframe = df_raw_source.copy()
outputframe["Vorname"] = df_list_vornamen
outputframe["Nachname"] = df_list_nachnamen
outputframe["Strasse"] = df_list_strassen
outputframe["Hausnummer"] = df_list_hausnummern
outputframe["EmailDOmains"] = df_list_emaildomains
outputframe["Telefonnummer"] = df_list_telefonnummer

In [236]:
# creates the email adresses with random maildomains
outputframe["Email"] = outputframe["Vorname"] + "." + outputframe["Nachname"] + "@" + outputframe["EmailDOmains"] 
del outputframe["EmailDOmains"]

In [237]:
outputframe.drop_duplicates() #removes duplicates that might be eventually be created (even if unlikely)
outputframe = outputframe.sample(frac=1).reset_index(drop=True)

In [238]:
print("Tabelle hat nun:{} Zeilen".format(outputframe.shape[0]))

Tabelle hat nun:999999 Zeilen


In [239]:
outputframe.head(100)

,Ort,PLZ,Gemeindename,Kanton,Sprache,Vorname,Nachname,Strasse,Hausnummer,Telefonnummer,Email
0,Genève,1203,Genève,GE,fr,Aleksandra,Christen,Eglistrasse,25,076/9242513,Aleksandra.Christen@bluewin.ch
1,Genève,1207,Genève,GE,fr,Fritz,Schwarz,Hohensteinweg,27,078/2669956,Fritz.Schwarz@yahoo.de
2,Zürich,8046,Zürich,ZH,de,Eduardo,Liechti,Frechenmätteliweg,168,076/8361609,Eduardo.Liechti@ntlworld.com
3,Luzern,6003,Luzern,LU,de,Dagmar,Weilenmann,Müseliweg,116,076/4923822,Dagmar.Weilenmann@live.ca
4,Genève,1204,Genève,GE,fr,Alfred,Reimann,Käshaldenstrasse,52,078/6675013,Alfred.Reimann@hotmail.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
95,Genève,1207,Genève,GE,fr,Tomasz,Gysin,Oetenbacherholzstrasse,116,077/9496279,Tomasz.Gysin@juno.com
96,Luzern,6004,Luzern,LU,de,Mariangela,Krüger,Ceresstrasse,18,078/5169292,Mariangela.Krüger@yahoo.es
97,Bern,3013,Bern,BE,de,Adnan,Lin,Flurstrasse,36,076/4359624,Adnan.Lin@bellsouth.net
98,Bern,3027,Bern,BE,de,Agron,Rösch,Traugottstrasse,25,078/7602297,Agron.Rösch@sbcglobal.net


In [240]:
# remove if you want a CSV Export
# outputframe.to_csv("Sample_data_leads.csv", sep=',', encoding='utf-8', index = False)